In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Ensure MarkLLM is in the path
sys.path.append('./MarkLLM')

from watermark.auto_watermark import AutoWatermark
from utils.transformers_config import TransformersConfig


In [2]:
cd MarkLLM


/home/cse/Amit-Zahid-Sahab/BanglaLLAMA-3-8b-Alpaca-orca-instruct/MarkLLM


In [ ]:
# !pip install -r requirements.txt


In [3]:
# Create __init__.py in watermark directory
open('watermark/__init__.py', 'a').close()

# Create __init__.py in utils directory
open('utils/__init__.py', 'a').close()


In [ ]:
# # !pip uninstall torch
# !pip install torch



In [4]:
import torch, random
import numpy as np

from transformers import AutoModelForCausalLM, AutoTokenizer
import tensorflow as tf

2025-04-24 13:21:21.638258: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-24 13:21:21.659788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-24 13:21:21.659808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-24 13:21:21.660449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 13:21:21.664461: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# Setting random seed for reproducibility
seed = 30
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"



In [ ]:
# pip install --upgrade transformers safetensors


In [ ]:
# pip install huggingface_hub

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# tokenizer=AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
tokenizer = AutoTokenizer.from_pretrained("BanglaLLM/BanglaLLama-3-8b-bangla-alpaca-orca-instruct-v0.0.1")
model = AutoModelForCausalLM.from_pretrained(
    "BanglaLLM/BanglaLLama-3-8b-bangla-alpaca-orca-instruct-v0.0.1",
    device_map={"": device},  # Load model to a specific device
    #torch_dtype=torch.float16,  # Use float16 for model weights to save memory
    load_in_8bit=True,  # Enable 8-bit quantization
    # Enable offloading of FP32 weights to CPU (optional)
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Transformers configuration for LLAMA-3-8b-instruct
transformers_config = TransformersConfig(
    model=model,
    tokenizer=tokenizer,
    vocab_size=tokenizer.vocab_size,  # Ensure the vocab size is correctly set
    device=model.device,
    max_new_tokens=500,
    min_length=50,
    do_sample=True,
    no_repeat_ngram_size=5
)

# Load watermarking algorithm )
if 'myWatermark' not in globals():
    myWatermark = AutoWatermark.load(
        'EXP',
        algorithm_config='config/EXP.json',
        transformers_config=transformers_config
    )


In [8]:
tokenizer.vocab_size

128000

In [ ]:

from transformers import pipeline


paraphraser = pipeline("text-generation", model=model,tokenizer=tokenizer)

#NEW

In [9]:
# Generate watermarked text
prompt = "বাংলাদেশের করোনা মোকাবেলায় পদক্ষেপ সম্পর্কে বলুন।"
 # Ensure that model and tensors are moved to the same device

# Tokenize the input and move the input tensors to the same device

# input_ids_ = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device) # Move input to device
watermarked_text = myWatermark.generate_watermarked_text(prompt)


In [10]:
detect_result_watermarked = myWatermark.detect_watermark(watermarked_text)

Observe model original output without watermarking

In [11]:
print("Watermarked Text: ", watermarked_text)
print("Detection Result for Watermarked Text: ", detect_result_watermarked)



Watermarked Text:  বাংলাদেশের করোনা মোকাবেলায় পদক্ষেপ সম্পর্কে বলুন। বাংলাদেশে করোনা ভাইরাস মোকাবেলায় প্রধানমন্ত্রীর সরকার গণভোটের মতো সমস্যার সমাধান করতে সংবিধান ব্যবহার করে একটি স্বীকৃত ও ন্যায্য প্রক্রিয়ার মাধ্যমে সিদ্ধান্ত নেও�
Detection Result for Watermarked Text:  {'is_watermarked': True, 'score': 1.892599918665104e-05}


In [12]:
unwatermarked_text = myWatermark.generate_unwatermarked_text(prompt)
detect_result_unwatermarked = myWatermark.detect_watermark(unwatermarked_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:

print("Unwatermarked Text: ", unwatermarked_text)
print("Detection Result for Unwatermarked Text: ", detect_result_unwatermarked)

Unwatermarked Text:  বাংলাদেশের করোনা মোকাবেলায় পদক্ষেপ সম্পর্কে বলুন। আমরা যে সমস্যার মুখোমুহ, আমাদের অবশ্যই এটি পরিচালনা করতে হবে।
আমি আশা করি আপনি এই কঠিন সমীক্ষা থেকে উপকৃত হব. আমি 2021 সালের মার্চ মাসে বাংগ্লাদে যাব এবং আমেরিকান দর্শনার্থীদের সাথে একটি ছোট গ্রুপের সঙ্গে যাত্রা করার পরিকল্পনা কারণ আমিও বাংরাদেসকে ভালবাসি। আপসবিটি আমন্ত্রণমূলক এব সহায়ক হব আম আশাকরি।
Detection Result for Unwatermarked Text:  {'is_watermarked': False, 'score': 0.716404537999322}


**Evaluation(21Feb,2025)**

In [18]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import textstat
import torch
from rouge import Rouge

In [19]:
dataset = load_dataset("BanglaLLM/bangla-alpaca")
samples = dataset["train"].shuffle(seed=42).select(range(10))  # Select 100 random samples


In [20]:
# Initialize evaluation lists
rouge = Rouge()
detection_results, perplexity_scores, rouge_scores = [], [], []
paraphrase_results, translation_results = [], []

In [21]:
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    return torch.exp(torch.tensor(loss))

In [ ]:
# Step 1: Generate watermarked & unwatermarked text
watermarked_texts, unwatermarked_texts = [], []
for sample in samples:
    prompt = sample["instruction"] + " " + sample["input"]
    watermarked_texts.append(myWatermark.generate_watermarked_text(prompt))
    unwatermarked_texts.append(myWatermark.generate_unwatermarked_text(prompt))

In [23]:
# Step 2: Detect watermark
for wm_text, unw_text in zip(watermarked_texts, unwatermarked_texts):
    detect_result_wm = myWatermark.detect_watermark(wm_text)
    detect_result_unw = myWatermark.detect_watermark(unw_text)
    detection_results.append((detect_result_wm["is_watermarked"], detect_result_unw["is_watermarked"]))


In [ ]:
for wm_text, unw_text in zip(watermarked_texts, unwatermarked_texts):
    print("Watermarked Text: ", wm_text)
    print("Unwatermarked Text: ", unw_text)

In [25]:
# Step 3: Calculate perplexity
for wm_text, unw_text in zip(watermarked_texts, unwatermarked_texts):
    perplexity_scores.append((calculate_perplexity(wm_text), calculate_perplexity(unw_text)))


In [26]:
# Step 4: Calculate ROUGE scores
rouge_scores = [rouge.get_scores(wm_text, unw_text)[0] for wm_text, unw_text in zip(watermarked_texts, unwatermarked_texts)]

In [ ]:
# # Step 5: Paraphrase Watermarked Text & Detect Watermark
# for wm_text in watermarked_texts:
#     paraphrased_text = paraphraser(wm_text, max_length=50, num_return_sequences=1)[0]['generated_text']
#     paraphrase_results.append(myWatermark.detect_watermark(paraphrased_text)["is_watermarked"])

**Rounttrip translation attack**

In [13]:
 !pip install git+https://github.com/csebuetnlp/normalizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-txwtg2fv
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-txwtg2fv
  git: 'credential-manager' is not a git command. See 'git --help'.
  git: 'credential-manager' is not a git command. See 'git --help'.
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,pipeline
from normalizer import normalize
# Load Bangla to English translation model
bn_en_model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_bn_en")
bn_en_tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_bn_en", use_fast=False)

# Load English to Bangla translation model
en_bn_model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_en_bn")
en_bn_tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_en_bn", use_fast=False)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
# Translate Bangla to English
def translate_bn_to_en(text):
    input_ids = bn_en_tokenizer(normalize(text), return_tensors="pt").input_ids
    generated_tokens = bn_en_model.generate(input_ids)
    return bn_en_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Translate English to Bangla
def translate_en_to_bn(text):
    input_ids = en_bn_tokenizer(normalize(text), return_tensors="pt").input_ids
    generated_tokens = en_bn_model.generate(input_ids)
    return en_bn_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [27]:
# Step 6: Round-Trip Translation & Detect Watermark
for wm_text in watermarked_texts:
    translated_text = translate_en_to_bn(translate_bn_to_en(wm_text))
    translation_results.append(myWatermark.detect_watermark(translated_text)["is_watermarked"])

In [16]:

translated_text = translate_en_to_bn(translate_bn_to_en(watermarked_text))
translation_result=myWatermark.detect_watermark(translated_text)
print("Translated Text: ", translated_text)
print("Detection Result for Translated Text: ", translation_result) 

Translated Text:  বাংলাদেশে করোনা ভাইরাস মোকাবেলায় বাংলাদেশের পদক্ষেপ নিয়ে আলোচনা। সংবিধান ব্যবহার করে গণভোটের মতো সমস্যা সমাধানের জন্য প্রধানমন্ত্রীর সরকার একটি স্বীকৃত ও ন্যায্য প্রক্রিয়ার মাধ্যমে সিদ্ধান্ত নেয়।
Detection Result for Translated Text:  {'is_watermarked': False, 'score': 0.0025497340001566803}


***Extra Part***

In [29]:
# Step 7: Compute & Display Final Metrics
detection_accuracy = sum([1 for result in detection_results if result[0] and not result[1]]) / len(detection_results)
avg_watermarked_perplexity = sum(p[0] for p in perplexity_scores) / len(perplexity_scores)
avg_unwatermarked_perplexity = sum(p[1] for p in perplexity_scores) / len(perplexity_scores)
avg_rouge = {
    "rouge-1": sum(score["rouge-1"]["f"] for score in rouge_scores) / len(rouge_scores),
    "rouge-2": sum(score["rouge-2"]["f"] for score in rouge_scores) / len(rouge_scores),
    "rouge-l": sum(score["rouge-l"]["f"] for score in rouge_scores) / len(rouge_scores),
}

In [ ]:
# paraphrase_robustness = sum(paraphrase_results) / len(paraphrase_results)
translation_robustness = sum(translation_results) / len(translation_results)

print("Detection Accuracy:", detection_accuracy)
print("Average Watermarked Perplexity:", avg_watermarked_perplexity)
print("Average Unwatermarked Perplexity:", avg_unwatermarked_perplexity)
print("Average ROUGE Scores:", avg_rouge)
# print("Robustness to Paraphrasing:", paraphrase_robustness)
print("Robustness to Round-Trip Translation:", translation_robustness)

**OLD**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert texts to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([watermarked_text,translated_text])


cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
print("Cosine Similarity:", cosine_sim[0][0])


Determining Token Difference

In [24]:

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

In [ ]:
tokens1 = tokenizer.encode(watermarked_text, add_special_tokens=False)
tokens2 = tokenizer.encode(translated_text, add_special_tokens=False)

num_tokens_text1 = len(tokens1)
num_tokens_text2 = len(tokens2)

# Difference in token counts
token_difference = abs(num_tokens_text1 - num_tokens_text2)

print(f"Number of tokens in Watermarked Text : {num_tokens_text1}")
print(f"Number of tokens in Retranslated Text: {num_tokens_text2}")
print(f"Difference in token counts: {token_difference}")

In [ ]:

tokens1_decoded = [tokenizer.decode([token_id]) for token_id in tokens1]
tokens2_decoded = [tokenizer.decode([token_id]) for token_id in tokens2]

# Print all tokens
print("Tokens in Watermarked Text:")
print(tokens1_decoded)

print("\nTokens in Retranslated Text:")
print(tokens2_decoded)

In [ ]:
print(tokenizer.vocab_size)  # Check the size of the vocabulary
print(tokenizer.get_vocab())  # Inspect the vocabulary

Number of similar tokens

In [ ]:
# Convert tokens to sets for comparison
set_tokens1 = set(tokens1)
set_tokens2 = set(tokens2)

# Find common tokens
common_tokens = set_tokens1.intersection(set_tokens2)

# Number of similar tokens
num_similar_tokens = len(common_tokens)

print(f"Number of similar tokens: {num_similar_tokens}")

print(len(set_tokens1))
print(len(set_tokens2))


#Visualization

In [ ]:
from visualize.font_settings import FontSettings
from visualize.visualizer import ContinuousVisualizer
from visualize.legend_settings import ContinuousLegendSettings
from visualize.page_layout_settings import PageLayoutSettings
from visualize.color_scheme import ColorSchemeForContinuousVisualization

In [ ]:
watermarked_data = myWatermark.get_data_for_visualization(watermarked_text)
unwatermarked_data = myWatermark.get_data_for_visualization(unwatermarked_text)

visualizer = ContinuousVisualizer(
    color_scheme=ColorSchemeForContinuousVisualization(),
    font_settings=FontSettings(),
    page_layout_settings=PageLayoutSettings(),
    legend_settings=ContinuousLegendSettings()
)

In [ ]:
watermarked_img = visualizer.visualize(
    data=watermarked_data,
    show_text=False,  # Show the text alongside the visualization
    visualize_weight=True,  # Visualize the watermark weights
    display_legend=True  # Display the legend to explain visualization
)

# Visualize the unwatermarked text with weights and the legend
unwatermarked_img = visualizer.visualize(
    data=unwatermarked_data,
    show_text=False,  # Show the unwatermarked text
    visualize_weight=True,  # Visualize the weights (important for watermark detection)
    display_legend=True  # Show the legend
)


In [ ]:
watermarked_img.save("EXP_watermarked.png")  # Save watermarked visualization
unwatermarked_img.save("EXP_unwatermarked.png")  # Save unwatermarked visualization

# Optionally, you can display the images directly (uncomment if you are using Jupyter/Colab)


In [ ]:
import matplotlib.pyplot as plt

# Display the watermarked image
plt.imshow(watermarked_img)
plt.axis('off')
plt.show()

plt.imshow(unwatermarked_img)
plt.axis('off')  # Hide axes for better visualization
plt.show()


#Evaluation

In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
cd./MarkLLM

In [ ]:

!gdown 1vPkJOVrtcyUQIpk6sIHYqV87z9eqCboF

In [ ]:
import pandas as pd

# Load the dataset from Google Drive
data = pd.read_csv('wiki.csv')

In [ ]:
print(data.iloc[0]['text'])
print(myWatermark.generate_watermarked_text(data.iloc[0]['text']) )


In [ ]:
from torch.nn.functional import cross_entropy

def calculate_perplexity(model, tokenizer, text, device):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    return torch.exp(torch.tensor(loss))

# Calculate perplexity for watermarked and unwatermarked text
watermarked_perplexity = calculate_perplexity(model, tokenizer, watermarked_text, device)
unwatermarked_perplexity = calculate_perplexity(model, tokenizer, unwatermarked_text, device)

print(f"Watermarked Text Perplexity: {watermarked_perplexity}")
print(f"Unwatermarked Text Perplexity: {unwatermarked_perplexity}")


In [ ]:
def distinct_ngrams(text, n):
    tokens = text.split()
    ngrams = set(zip(*[tokens[i:] for i in range(n)]))
    return len(ngrams) / len(tokens)

watermarked_div1 = distinct_ngrams(watermarked_text, 1)
watermarked_div2 = distinct_ngrams(watermarked_text, 3)

unwatermarked_div1 = distinct_ngrams(unwatermarked_text, 1)
unwatermarked_div2 = distinct_ngrams(unwatermarked_text, 3)

print(f"Watermarked Text Distinct-1: {watermarked_div1}, Distinct-2: {watermarked_div2}")
print(f"Unwatermarked Text Distinct-1: {unwatermarked_div1}, Distinct-2: {unwatermarked_div2}")


In [ ]:
!pip install textstat


In [ ]:
import textstat

watermarked_readability = textstat.flesch_reading_ease(watermarked_text)
unwatermarked_readability = textstat.flesch_reading_ease(unwatermarked_text)

print(f"Watermarked Text Readability: {watermarked_readability}")
print(f"Unwatermarked Text Readability: {unwatermarked_readability}")


In [ ]:
test="দশম জাতীয় সংসদ নির্বাচন ভালোভাবে হয়নি। এর পেছনের কারণ অন্যরকম ছিল। এই নির্বাচন তাদের না জানা কিছু ব্যাপার ছিল, যা তারা বলেনি বা কোন আইন মানার দরকার মনে করেনি। আসলে এর পেছনে অন্য প্রার্থী তাদের সম্পদের হিসাব না দেওয়ার কারণে বা অন্য সমস্যা ছিল।"

In [ ]:
detect_result_watermarked = myWatermark.detect_watermark(test)
detect_result_watermarked

In [ ]:
from transformers import pipeline
paraphraser = pipeline("text2text-generation", model="BanglaLLM/BanglaLLama-3-8b-bangla-alpaca-orca-instruct-v0.0.1", device=model.device)  # Use GPU if available (device=0)
# Generate a paraphrase with adjusted settings
paraphrased_text = paraphraser(
    watermarked_text,
    max_new_tokens=20,  # Allows generating up to 50 new tokens
    num_beams=3,  # Uses beam search for higher quality paraphrasing
    do_sample=True  # Adds randomness to make paraphrasing more diverse
)[0]['generated_text']

# Print paraphrased text
print("Paraphrased Text:", paraphrased_text)


In [ ]:
detect_result_watermarked = myWatermark.detect_watermark(paraphrased_text)
detect_result_watermarked

Try Semantic embedding generation with Sentence piece transformer

In [17]:
from sentence_transformers import SentenceTransformer

In [18]:
embedding_model = SentenceTransformer("shihab17/bangla-sentence-transformer")

In [19]:
sentences = [
    watermarked_text,
    unwatermarked_text,
    translated_text
]

In [20]:
embeddings = embedding_model.encode(sentences)


In [21]:
# Compute Similarities
similarity_watermarked_unwatermarked = torch.nn.functional.cosine_similarity(
    torch.tensor(embeddings[0]), torch.tensor(embeddings[1]), dim=0
).item()

similarity_watermarked_retranslated = torch.nn.functional.cosine_similarity(
    torch.tensor(embeddings[0]), torch.tensor(embeddings[2]), dim=0
).item()

In [22]:
print("\nSemantic Similarity (Watermarked vs Unwatermarked):", similarity_watermarked_unwatermarked)
print("Semantic Similarity (Watermarked vs Retranslated Watermarked):", similarity_watermarked_retranslated)


Semantic Similarity (Watermarked vs Unwatermarked): 0.5899903774261475
Semantic Similarity (Watermarked vs Retranslated Watermarked): 0.943682849407196


In [ ]:
# Print the embeddings
print("\nEmbeddings:")
for i, sentence in enumerate(sentences):
    print(f"\nText {i+1}: {sentence}")
    print(f"Embedding: {embeddings[i]}")